In [50]:
import pandas as pd
import networkx as nx
import statistics

from debugpy import connect

In [51]:
graph = "swtor"
max_community_size = 20
connected_percent = 0.75
times_average = 1.5

In [52]:
print("Reading graph " + str(graph))
graph_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples_weighted/"
graph_df = pd.read_csv(graph_path + graph + ".triples", sep="###", engine="python", header=None)
G = nx.Graph()
for row in graph_df.itertuples():
    G.add_edge(int(row[1]), int(row[2]), weight=int(row[3]))

Reading graph swtor


In [53]:
print(G)
print([n for n in G[0]])

Graph with 18125 nodes and 57118 edges
[2, 2180]


In [54]:
mean_edge_weight = statistics.mean(weight for _, _, weight in G.edges(data='weight'))
print(mean_edge_weight)

1.3045449770650233


In [55]:
communities_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/communities/"
communities = list({i} for i in G.nodes)

print("Parameters - connected_percent: " + str(connected_percent) + " - times_average: " + str(times_average))
print("Searching for communities in graph " + graph)

for community_size in range(2, max_community_size + 1):
    print("Community size: " + str(community_size))
    last_iteration = 0
    to_add = list()
    to_delete = list()
    for community in communities:
        if len(community) != community_size - 1:
            continue
        neighbors = set()
        for node in community:
            neighbors.update(G.neighbors(node))
            # print(node)
            # print(neighbors)
        neighbors = [item for item in neighbors if item not in community]
        # print(neighbors)
        for node in neighbors:
            community_temp = list(community.copy())
            community_temp.append(node)
            H = G.subgraph(community_temp)
            # print(community)
            # print(community_temp)
            # print(statistics.mean(weight for _, _, weight in H.edges(data='weight')))
            # print(len([h for h in H.neighbors(node)]))
            if statistics.mean(weight for _, _, weight in H.edges(data='weight')) \
                    > mean_edge_weight * times_average and \
                    len([h for h in H.neighbors(node)]) >= (community_size - 1) * connected_percent:
                # print("old: " + str(community))
                # print("new: " + str(community_temp))
                last_iteration += 1
                if community not in to_delete:
                    to_delete.append(community)
                if set(community_temp) not in to_add and set(community_temp) not in communities:
                    to_add.append(set(community_temp)) 
            if community_size == 2:
                communities = [item for item in communities if len(item) != 1]
    if last_iteration == 0:
        print("Break at community size: " + str(community_size))
        break
    # print(to_add)
    # print(to_delete)
    communities = [item for item in communities if item not in to_delete]
    communities.extend(to_add)
    print(len(communities))

# print(communities)

with open(communities_path + graph + ".csv", 'w') as communities_output:
    for community in communities:
        communities_output.write(str(community)
                                 .replace("{", "")
                                 .replace("}", "")
                                 .replace(", ", ";") + "\n")

print("Communities for graph " + graph + " found")

Parameters - connected_percent: 0.75 - times_average: 1.5
Searching for communities in graph swtor
Community size: 2
16207
Community size: 3
14352
Community size: 4
13387
Community size: 5
13707
Community size: 6
13505
Community size: 7
13362
Community size: 8
13359
Community size: 9
13357
Community size: 10
Break at community size: 10
Communities for graph swtor found
